In [ ]:
from igraph import *
import random

In [ ]:
def getPermutation(graph):
    permut = []
    for x in range(graph.vcount()):
        permut.append(x)
    random.shuffle(permut)
    random.shuffle(permut)
    return permut

In [ ]:
def GetWeightSum(g):
    sum = 0
    for x in g.es:
        sum += x["weight"]
    return sum

def GetWeightSumInCommunity(g,communityNum):
    sum = 0
    vertices = g.vs.select(lambda v: v["community"]== communityNum)
    for x in vertices:
        for y in vertices:
            sum+=g[x,y]
    return sum

def GetWeightOfLinksToKInCommunity(g,k,communityNum):
    sum = 0
    vertices = g.vs.select(lambda v: v["community"]== communityNum)
    for v in vertices:
        if k!=v.index:
            sum += g[k,v]
    return sum

def GetWeightOfLinksToCommunity(g,communityNum):
    sum = 0
    param = 0.75 # interesting stuff
    vertices = g.vs.select(lambda v: v["community"]== communityNum)
    for v in vertices:
        sum += GetWeightedDegree(g,v)
    return sum * param

def GetWeightedDegree(g,k):
    sum = 0
    for y in g.neighbors(k):
        sum += g[k,y] 
    return sum

In [ ]:
def GetModularityChange2(g,i,community):
    change = 0
    value1 = (GetWeightSumInCommunity(g,community) + GetWeightOfLinksToKInCommunity(g,i,community)*2)/(2*WeightSum(g)) - (( (GetWeightOfLinksToCommunity(g,i,community) + GetWeightedDegree(g,i))/(2*GetWeightSum(g)))**2)
    value2 = (GetWeightSumInCommunity(g,community))/(2*GetWeightSum(g)) - (GetWeightOfLinksToCommunity(g,i,community)/(2*GetWeightSum(g)))**2 - (GetWeightedDegree(g,i)/(2*GetWeightSum(g)))**2
    return value1-value2

def GetModularityChange(g,i,community):
    change=0
    m = GetWeightSum(g)
    value1= GetWeightOfLinksToKInCommunity(g,i,community)/m - (GetWeightOfLinksToCommunity(g,community)*GetWeightedDegree(g,i))/(2*(m**2))
    return value1


In [ ]:
def ChangeCommunity(comtable,comfrom,comto):
    i=0
    for x in comtable:
        if x == comfrom:
            comtable[i] = comto
        i += 1
def GetIndices(comtable,com):
    l = []
    i = 0
    for x in comtable:
        if x == com:
            l.append(i)
        i += 1
    return l

In [ ]:
def GetVerticesInCommunity(g,com):
    vertices = g.vs.select(lambda v: v["community"]== com)
    return [v.index for v in vertices]

def GetWeightOfLinksBetweenCommunities(g,com1,com2):
    vert1 = GetVerticesInCommunity(g,com1)
    vert2 = GetVerticesInCommunity(g,com2)
    ret = 0
    if(com1 != com2):
        for v1 in vert1:
            for v2 in vert2:
                ret += g[v1,v2]
    else:
        ret = GetWeightSumInCommunity(g,com1)
    return ret

In [ ]:
def MergeCommunities(g):
    newgraph = Graph()
    for com1 in set(g.vs["community"]):
        newgraph.add_vertex(community =com1)
    i=0
    communities = newgraph.vs["community"]
    for com1 in communities:
        for com2 in communities[i:]:
            weight=GetWeightOfLinksBetweenCommunities(g,com1,com2)
            if weight>0:
                es = newgraph.add_edge([v["community"] for v in newgraph.vs].index(com1),[v["community"] for v in newgraph.vs].index(com2))
                es["weight"]=weight
        i+=1
        
    return newgraph

In [ ]:
def LouvainIteration(g,permut,comtable):
    changedtable = comtable.copy()
    waschange = False
    for x in range(g.vcount()):
        prevcom = g.vs[permut[x]]["community"]
        maxmodchange = 0.0
        maxcommunity = -1
        communities = {}
        for v in g.neighbors(permut[x]):
            if  g.vs[v]["community"] in communities:
                continue
            communities[g.vs[v]["community"]]=1
            g.vs[permut[x]]["community"] = g.vs[v]["community"]

            modchange = GetModularityChange(g,permut[x],g.vs[v]["community"])

            if(maxmodchange < modchange):
                maxmodchange = modchange
                maxcommunity = g.vs[v]["community"]
           
        g.vs[permut[x]]["community"] = prevcom

        if maxcommunity != -1 and maxcommunity != prevcom:
            waschange=True
            g.vs[permut[x]]["community"] = maxcommunity
            for i in GetIndices(comtable,prevcom):
                changedtable[i] = maxcommunity
    comtable[:] = changedtable[:]
    return waschange

In [ ]:
def LouvainAlogrithm(g):
    graph=g.copy()
    for x in range(graph.vcount()):
        graph.vs[x]["community"]=x
    comtable = graph.vs["community"]
    permut = getPermutation(graph)
    flag=True
    while flag:
        flag = LouvainIteration(graph,permut,comtable)
        graph = MergeCommunities(graph)
        permut = getPermutation(graph)

    return comtable

In [ ]:
def GetModularity(g):
    m = GetWeightSum(g)
    sum = 0
    for i in range(g.vcount()):
        for j in range(g.vcount()):
            if(g.vs[i]["community"] == g.vs[j]["community"]):
                sum += (g[i,j]-((GetWeightedDegree(g,i)*GetWeightedDegree(g,j))/(2*m)))
    mod = sum /(2*m)
    return mod

In [ ]:
def plotsubgraph(g,vertices):
    subg = g.subgraph(vertices)
 
    comtable=LouvainAlogrithm(subg)
    for v in subg.vs:
        v["communities"] = []
        v["communities"].append(comtable[v.index])
    plot_full(subg, "full.svg")
    i=0
    for v in subg.vs:
        v["community"] = comtable[i]
        i+=1
    print(GetModularity(subg))

In [ ]:
def GetOverlapPercent(indices1, indices2):
    common = list(set(indices1).intersection(indices2))
    if len(indices1) > len(indices2):
        return len(common) / len(indices1)
    else:
        return len(common) / len(indices2)

def AssignCommunities(originaltable, currenttable):
    newtable = [0] * len(currenttable)
    for x in range(len(currenttable)):
        indices = GetIndices(currenttable,x)
        if len(indices)==0:
            continue
        bestfittingcom=-1
        bestoverlap=0
        for y in range(len(originaltable)):
            indices2 = GetIndices(originaltable,y)
            if len(indices2)==0:
                continue
            if GetOverlapPercent(indices,indices2)>bestoverlap:
                bestoverlap=GetOverlapPercent(indices,indices2)
                bestfittingcom=y
        if(bestfittingcom != -1):
            indices = GetIndices(currenttable,x)
            for i in indices:
                newtable[i]=bestfittingcom
    return newtable

In [ ]:
def RearrangeCommunties(comtable):  
    counter=0
    newtable= comtable.copy()
    for i in range(len(newtable)):
        if i not in newtable:
            for n in range(len(newtable))[i+1:]:
                if n in newtable:
                    indices = GetIndices(newtable,n)
                    for x in indices:
                        newtable[x]=i
                    counter += len(indices)
                    break
        if(counter == len(comtable)):
            break
    return newtable

In [ ]:
def UpdateCommunityMatrix(g,belongingtable,comtable):
    for x in range(g.vcount()):
          for y in g.neighbors(x):
              belongingtable[x][comtable[y]]+=1

In [ ]:
def NormalizeBelonging(belongingtable):
    for x in range(len(belongingtable)):
        tablesum=sum(belongingtable[x])
        if tablesum ==0:
            continue
        for y in range(len(belongingtable[x])):
            belongingtable[x][y]=belongingtable[x][y]/tablesum
    return belongingtable

In [ ]:
def DivideIntoCommunities(g,ov,belongingtable):
    lambd=1/(ov+1)
    for x in range(g.vcount()):
        g.vs[x]["communities"]=[]
        if sum(belongingtable[x]) == 0:
             g.vs[x]["communities"].append(x)
             continue
        for y in range(len(belongingtable[x])):
            if belongingtable[x][y]>= lambd:
                g.vs[x]["communities"].append(y+1)